In [2]:
import fiftyone as fo
import glob
from pycocotools.coco import COCO

# img_list = glob.glob('../dataset/defect/*') +  glob.glob('../dataset/normal/*')
# dataset = fo.Dataset.from_images(img_list)

coco = COCO('../info/test.json')
img_list = ['../dataset/' + anno['file_name'] for anno in coco.loadImgs(coco.getImgIds())]
# Create a dataset from a glob pattern of images
dataset = fo.Dataset.from_images(img_list)

# if __name__=="__main__":
#     dataset = fo.Dataset.from_images(img_list)
#     session = fo.launch_app(dataset , port =5000 , address= "0.0.0.0" )
#     session.wait()
import json
img_id = coco.getImgIds()[11]
coco.getAnnIds(img_id)
coco.loadCats(coco.getCatIds())

id_to_cat = {v['id'] : v['name'] for  v in coco.loadCats(coco.getCatIds()) }

with open('../mmdetection/result/retinanet_swin-l/result.bbox.json' , 'r') as f:
    data = json.load(f)

samples = []
for img_id in coco.getImgIds():
    sample = fo.Sample(filepath= '../dataset/' + coco.loadImgs(img_id)[0]['file_name'])

    detections = []
    for ann_id in coco.getAnnIds(imgIds = img_id):
        obj = coco.loadAnns(ann_id)[0]
        label = id_to_cat[obj['category_id']]
        bounding_box = obj['bbox']

        detections.append(
            fo.Detection(label = label , bounding_box = bounding_box)
        )

    predictions_anno = [data_i for data_i in data if data_i['image_id'] == img_id]
    predictions = []
    for pred in predictions_anno:
        pred['image_id']
        pred['bbox']
        pred['score']
        predictions.append(
            fo.Detection(label = id_to_cat[pred['category_id']] , bounding_box = pred['bbox'] , confidence = pred['score'])
        )
        
    if detections:
        sample['ground_truth'] = fo.Detections(detections = detections)
        samples.append(sample)

    if predictions:
        sample['prediction'] = fo.Detections(detections = predictions)
        samples.append(sample)

dataset = fo.Dataset('my-detection-dataset')
dataset.add_samples(samples)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
 100% |█████████████████| 472/472 [104.0ms elapsed, 0s remaining, 4.5K samples/s]  
